<a href="https://colab.research.google.com/github/haikuism/flashing-heart/blob/master/%E7%8C%AB%E7%8A%AC%E8%B1%9A_%E7%94%BB%E5%83%8F%E5%88%86%E9%A1%9E%E3%83%97%E3%83%AD%E3%82%B8%E3%82%A7%E3%82%AF%E3%83%88_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
インターネットから猫、犬、豚の画像をそれぞれ11枚ずつ収集し、各クラス10枚を学習用、1枚をテスト用としてディープラーニングモデルを構築・学習・評価し、テスト結果を正誤で表示する。

## 画像の収集

### Subtask:
猫、犬、豚の画像をそれぞれ11枚ずつインターネットから収集します。


## データセットの準備

### Subtask:
収集した画像を学習用（各クラス10枚）とテスト用（各クラス1枚）に分割します。


**Reasoning**:
I need to organize the collected images into training and testing sets. I will first define the paths to the image directories and then iterate through each class to move one random image to the test directory while keeping the rest in the training directory. Finally, I will verify the directory structure.



In [ ]:
import os
import random
import shutil

# Define the base directory where the images are stored (assuming images are in 'images' folder)
base_dir = 'images'
classes = ['cat', 'dog', 'pig']
train_dir = 'train'
test_dir = 'test'

# Create train and test directories with class subdirectories
for dir_path in [train_dir, test_dir]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    for class_name in classes:
        class_dir = os.path.join(dir_path, class_name)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

# Move 1 image from each class to the test directory
for class_name in classes:
    source_dir = os.path.join(base_dir, class_name)
    image_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]
    if len(image_files) != 11:
        print(f"Warning: Expected 11 images in {source_dir}, but found {len(image_files)}")
        # This is a warning, we will still attempt to split if there are enough images
        if len(image_files) < 1:
             print(f"Error: No images found in {source_dir}. Skipping this class.")
             continue

    # Select one random image for testing
    test_image = random.choice(image_files)
    image_files.remove(test_image) # Remove the test image from the list

    # Move the test image
    src_path = os.path.join(source_dir, test_image)
    dest_path = os.path.join(test_dir, class_name, test_image)
    shutil.move(src_path, dest_path)

    # Move the remaining 10 images to the training directory
    for train_image in image_files:
        src_path = os.path.join(source_dir, train_image)
        dest_path = os.path.join(train_dir, class_name, train_image)
        shutil.move(src_path, dest_path)

# Verify the directory structure and counts
print("\nVerification:")
for dir_path in [train_dir, test_dir]:
    print(f"\nContents of {dir_path}:")
    for class_name in classes:
        class_dir = os.path.join(dir_path, class_name)
        if os.path.exists(class_dir):
            file_count = len([f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))])
            print(f"  {class_name}: {file_count} images")
        else:
            print(f"  {class_name}: Directory not found")


## モデルの構築

### Subtask:
ディープラーニングモデルを構築します。シンプルなCNNモデルを検討します。


**Reasoning**:
Import necessary modules, define the input shape, create a Sequential model, add layers, compile the model, and print the summary as instructed.



### Subtask:
インターネットから猫の画像を収集します。以下のコードは、指定したURLリストから画像をダウンロードします。ダウンロードしたい画像のURLリストを準備してください。

In [ ]:
import os

# 画像を保存するディレクトリを作成
image_dir = 'images/cat'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

# ダウンロードしたい猫の画像のURLリスト
# 例: 以下のリストを実際にダウンロードしたい画像のURLに置き換えてください。
image_urls = [
     'https://www.npf.co.jp/cms/npfmagazine/uploads/2023/10/3727647_s.jpg',
'https://www.npf.co.jp/cms/npfmagazine/uploads/2023/10/23978637_s.jpg',
'https://www.mfr.co.jp/content/dam/mfrcojp/sumai/chishiki_018/img/s302-key.jpg',
'https://cdn.shopify.com/s/files/1/0551/9897/4030/files/cat010_1-780x1024.jpg',
'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQgtBlzd4XzU3c20yhOMgy2fj0_OluHFjdRcw&s',
'https://page.mkgr.jp/ownedmedia/wordpress/wp-content/uploads/2023/11/image1-1.jpg',
'https://page.mkgr.jp/ownedmedia/wordpress/wp-content/uploads/2023/11/image4-2-768x511.jpg',
'https://www.npf.co.jp/cms/npfmagazine/uploads/2023/10/27750021_m.jpg',
'https://www.npf.co.jp/cms/npfmagazine/uploads/2023/10/196844355_s.jpg',
'https://www.npf.co.jp/cms/npfmagazine/uploads/2023/10/4443649_s.jpg',
     'https://www.npf.co.jp/cms/npfmagazine/uploads/2023/10/22364658_s.jpg'
    # ... 10枚分のURLを追加
]

# URLから画像をダウンロード
for i, url in enumerate(image_urls):
    # ダウンロードするファイルのパスを生成
    filename = os.path.join(image_dir, f'cat_{i+1}.jpg')
    # wget コマンドを使用してファイルをダウンロード
    # -q は静かに（Quiet）ダウンロードするためのオプションです
    os.system(f'wget -O {filename} {url} -q')

print(f"{len(image_urls)} images downloaded to {image_dir}")

11 images downloaded to images/cat


### Subtask:
ダウンロードした猫の画像ファイルを確認します。

In [ ]:
import os

# 画像を保存したディレクトリのパス
image_dir = 'images/cat'

# ディレクトリ内のファイル一覧を取得
if os.path.exists(image_dir):
    image_files = os.listdir(image_dir)
    print(f"Directory: {image_dir}")
    print(f"Number of files: {len(image_files)}")
    print("Files:")
    for file in image_files:
        print(f"- {file}")
else:
    print(f"Directory not found: {image_dir}")

Directory: images/cat
Number of files: 11
Files:
- cat_4.jpg
- cat_5.jpg
- cat_11.jpg
- cat_1.jpg
- cat_8.jpg
- cat_7.jpg
- cat_10.jpg
- cat_2.jpg
- cat_3.jpg
- cat_6.jpg
- cat_9.jpg


### Subtask:
ダウンロードした猫の画像をいくつか表示して確認します。

In [ ]:
import os
from IPython.display import display, Image

# 画像を保存したディレクトリのパス
image_dir = 'images/cat'

# ディレクトリ内のファイル一覧を取得
if os.path.exists(image_dir):
    image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
    print(f"Displaying a few images from {image_dir}:")

    # 最初の数枚の画像を表示 (例えば5枚)
   # num_images_to_display = min(11, len(image_files)) # ファイル数に応じて調整


        image_path = os.path.join(image_dir, image_files[1])
        print(f"Displaying: {image_files[1]}")
        display(Image(filename=image_path))


IndentationError: unexpected indent (ipython-input-3918614151.py, line 16)

In [ ]:
import os

image_dir = 'images/cat'
full_path = os.path.abspath(image_dir)
print(f"The full path to '{image_dir}' is: {full_path}")

IndentationError: unexpected indent (ipython-input-3645865094.py, line 5)